In [1]:
!pip install PyMuPDF python-docx transformers sentence-transformers pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip install PyMuPDF
import fitz  # PyMuPDF

In [3]:
!pip install python-docx
from docx import Document

In [4]:
import os
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone with API key
api_key = "PINECONE_API_KEY"
pc = Pinecone(api_key=api_key)

# Define index parameters
index_name = "document-embeddings"
dimension = 768  # Embedding dimension (e.g., for BERT-like models)

# Create the index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric='cosine',  # You can also use 'euclidean'
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'  # Your preferred region
        )
    )

# Connect to the index using `pc.Index()`
index = pc.Index(index_name)

In [5]:
import torch

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [6]:
def extract_text_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    for page in doc:
        text += page.get_text()
    return text
def extract_text_from_docx(docx_file):
    doc = Document(docx_file)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text

def extract_text_from_txt(txt_file):
    with open(txt_file, 'r') as file:
        text = file.read()
    return text

In [7]:
from sentence_transformers import SentenceTransformer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 5. Load model and move to GPU if available
model = SentenceTransformer('all-MiniLM-L12-v2')
model = model.to(device)

Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
def tokenize_and_embed(text):
    sentences = text.split(". ")
    embeddings = model.encode(sentences, show_progress_bar=True, device=device)
    return sentences, embeddings
def upload_to_pinecone(sentences, embeddings, index):
    max_metadata_length = 40000
    for i, (sentence, embedding) in enumerate(zip(sentences, embeddings)):
        # Truncate the sentence if it's too long
        truncated_sentence = sentence[:max_metadata_length]
        index.upsert([(f"sentence-{i}", embedding.tolist(), {"text": truncated_sentence})])


In [9]:
!git clone https://github.com/ranjana-9/LawgorithmsATMECS.git



Cloning into 'LawgorithmsATMECS'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 71 (delta 15), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (71/71), 28.36 MiB | 11.41 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [10]:
import os

# Path to your cloned repository
repo_path = "/content/LawgorithmsATMECS"

# Initialize lists for different file types
pdf_files = []
docx_files = []
txt_files = []

# Walk through the repository and collect relevant files
for root, dirs, files in os.walk(repo_path):
    for file in files:
        if file.endswith(".pdf"):
            pdf_files.append(os.path.join(root, file))
        elif file.endswith(".docx"):
            docx_files.append(os.path.join(root, file))
        elif file.endswith(".txt"):
            txt_files.append(os.path.join(root, file))

# Print the collected file paths
print("PDF files:", pdf_files)
print("DOCX files:", docx_files)
print("TXT files:", txt_files)


PDF files: ['/content/LawgorithmsATMECS/general_data/Companies_Act_1956_13jun2011-1.pdf', '/content/LawgorithmsATMECS/general_data/LLP ACT.pdf', '/content/LawgorithmsATMECS/general_data/Companies_Act_1956_13jun2011-2.pdf', '/content/LawgorithmsATMECS/corporate laws/corporate laws-3.pdf', '/content/LawgorithmsATMECS/corporate laws/corporate laws-6.pdf', '/content/LawgorithmsATMECS/corporate laws/corporate laws-1.pdf', '/content/LawgorithmsATMECS/corporate laws/corporate laws-4.pdf', '/content/LawgorithmsATMECS/corporate laws/corporate laws-2.pdf', '/content/LawgorithmsATMECS/corporate laws/corporate laws-5.pdf', '/content/LawgorithmsATMECS/corporate laws/corporate laws-7.pdf', '/content/LawgorithmsATMECS/company_data/SAMPLE_Patent.pdf', '/content/LawgorithmsATMECS/company_data/SAMPLE_SLA.pdf', '/content/LawgorithmsATMECS/company_data/SAMPLE_Shareholder_Agreement.pdf']
DOCX files: ['/content/LawgorithmsATMECS/company_data/SAMPLE_NDA.docx', '/content/LawgorithmsATMECS/company_data/SAMPLE_

In [ ]:
# Function to process all files
def process_files(file_list, extract_text_function, tokenize_and_embed_function, pinecone_index):
    for file_path in file_list:
        # Extract text
        text = extract_text_function(file_path)
        # Tokenize and embed
        sentences, embeddings = tokenize_and_embed_function(text)
        # Upload to Pinecone
        upload_to_pinecone(sentences, embeddings, pinecone_index)

# Process PDF files
process_files(pdf_files, extract_text_from_pdf, tokenize_and_embed, index)

# Process DOCX files
process_files(docx_files, extract_text_from_docx, tokenize_and_embed, index)

# Process TXT files
process_files(txt_files, extract_text_from_txt, tokenize_and_embed, index)


Batches:   0%|          | 0/141 [00:00<?, ?it/s]

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Batches:   0%|          | 0/121 [00:00<?, ?it/s]

In [ ]:
import os

# Path to your cloned repository
repo_path = "/content/LawgorithmsATMECS"  # Adjust this to your cloned repo path

# Function to list all folders in a given directory
def list_folders(directory):
    folders = []
    for root, dirs, files in os.walk(directory):
        # Append the current directory to the folders list
        folders.append(root)
        # Break the loop after listing the top-level folders
        break
    return folders[0], dirs  # Return the root and its directories

# Get the root and directories
root, subfolders = list_folders(repo_path)

# Print the root and available subfolders
print(f"Root directory: {root}")
print("Available folders:")
for folder in subfolders:
    print(f"- {folder}")


Root directory: /content/LawgorithmsATMECS
Available folders:
- cases
- company_data
- .git
- general_data
- corporate laws
